In [1]:
import nltk
nltk.download('punkt_tab')

!pip install requests beautifulsoup4 nltk pandas tqdm scikit-learn transformers torch spacy simalign pandas numpy matplotlib

!pip install indic-nlp-library

!python -m spacy download xx_ent_wiki_sm


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
###############################################################################
# FULL SCRIPT  – single‑pass crawl + multi‑threshold export
###############################################################################

import nltk
nltk.download('punkt')

import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import spacy
import logging
import numpy as np
from simalign import SentenceAligner
from indicnlp.tokenize import sentence_tokenize
from indicnlp import common
import json
import os
import time
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ------------------------------------------------------------------ #
#  Polite, retry‑aware HTTP session for the Wikimedia API
# ------------------------------------------------------------------ #
session = requests.Session()
session.headers.update({
    "User-Agent": "WikiMiner/1.0 (eaa1.17037505@gmail.com)"  # <-- put real email
})
retry = Retry(
    total=5,                # up to 5 retries
    backoff_factor=1.5,     # sleep: 1.5s, 3s, 4.5s, ...
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount("https://", HTTPAdapter(max_retries=retry))
session.mount("http://",  HTTPAdapter(max_retries=retry))
# ------------------------------------------------------------------ #


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

###############################################################################
#                              CONFIGURATION
###############################################################################

INDIC_LANGUAGES = [
    'as', 'bn', 'gu', 'hi', 'kn', 'ml', 'mr', 'or', 'pa', 'ta', 'te',
]

LANG_CODE_MAP = {
    'as': 'Assamese', 'bn': 'Bengali', 'gu': 'Gujarati', 'hi': 'Hindi',
    'kn': 'Kannada',  'ml': 'Malayalam', 'mr': 'Marathi', 'or': 'Oriya',
    'pa': 'Punjabi',  'ta': 'Tamil',     'te': 'Telugu',
}

# Define categories to crawl in English, which (hopefully) contain articles relevant
# to each language. You can customize or expand these categories as needed.
language_categories = {
    'as': [
        "Articles containing Assamese-language text",
        "Assamese-language actors",
        "People from Assam",
        "Assam Valley Literary Award",
        "Assamese singers",
        "Novels in Assamese",
        "CS1 Assamese-language sources (as)",
        "Assamese-language film stubs",
        "Assamese-language mass media",
        "Assamese people",
        "Assamese nationalism",
        # Newly added categories for Assamese
        "Assamese literature",
        "Cinema of Assam",
        "Music of Assam",
        "Festivals in Assam",
        "Assamese cuisine",
        "Assamese folklore & mythology",
        "Dances and theatre of Assam",
        "Assamese-language media",
        "Notable Assamese people",
        "Awards and honors in Assam",
    ],
    'bn': [
        "Bengali-language films",
        "Bengali people",
        "Bengali-language writers",
        "Bengali words and phrases",
        "Bengali language",
        "Articles containing Bengali-language text",
        "Bengali poetry in English translation",
        "Songs in Bengali",
        "Bengali nationalism",
        "Bengali-language surnames",
        # Newly added categories for Bengali
        "Bengali literature",
        "Bengali cinema",
        "Bengali music and songs",
        "Festivals of Bengal",
        "Bengali cuisine",
        "Bengali mythology & folk tales",
        "Bengali theatre and dance",
        "Notable Bengali people",
        "Literary awards in Bengali",
        "Bengali media",
    ],
    'gu': [
        "Gujarati people",
        "Gujarati writers",
        "Articles containing Gujarati-language text",
        "Gujarati literature",
        "Gujarati-language films",
        "Gujarati-language books",
        "Gujarati-language writers",
        "Gujarati language",
        "Articles with Gujarati-language sources (gu)",
        "Gujarati-language songs",
        # Newly added categories for Gujarati
        "Gujarati cinema",
        "Gujarati music",
        "Festivals in Gujarat",
        "Gujarati cuisine",
        "Gujarati folklore & mythology",
        "Gujarati theatre (Bhavai)",
        "Gujarati media",
        "Notable Gujarati people",
        "Cultural institutions & awards",
    ],
    'hi': [
        "Hindi",
        "Hindi-language films",
        "Actors in Hindi cinema",
        "Hindi-language literature",
        "Hindi-language film directors",
        "Hindi-language writers",
        "Articles containing Hindi-language text",
        "Hindi theatre",
        "Hindi-language poets",
        # Newly added categories for Hindi
        "Hindi literature",
        "Hindi cinema (Bollywood)",
        "Hindi music and songs",
        "Indian festivals (North India)",
        "Hindu epics and mythology",
        "Hindi media",
        "Notable Hindi authors and poets",
        "Hindi theatre and folk arts",
        "National awards (Hindi context)",
        "Historical events (Hindi belt)",
    ],
    'kn': [
        "Kannada-language films",
        "Kannada poets",
        "Kannada playback singers",
        "Kannada film directors",
        "Kannada literature",
        "Kannada cinema",
        "Kannada grammar",
        "Kannada-language writers",
        "Articles containing Kannada-language text",
        "Kannada people",
        # Newly added categories for Kannada
        "Kannada music",
        "Festivals of Karnataka",
        "Kannada cuisine",
        "Kannada folklore & mythology",
        "Kannada theater and dance",
        "Notable Kannada people",
        "Awards in Kannada",
        "Kannada media",
    ],
    'ml': [
        "Malayalam-language films",
        "Malayalam cinema",
        "Articles containing Malayalam-language text",
        "Malayalam poets",
        "Malayalam-language writers",
        "Malayalam grammar",
        "Malayalam-language newspapers",
        "Malayalam language",
        "Malayalam-language novelists",
        # Newly added categories for Malayalam
        "Malayalam literature",
        "Malayalam music",
        "Festivals of Kerala",
        "Malayalam cuisine",
        "Kerala folklore & mythology",
        "Malayalam theatre and dance",
        "Notable Malayalam people",
        "Malayalam media",
        "Kerala history events",
    ],
    'mr': [
        "Marathi language",
        "Marathi people",
        "Marathi-language films",
        "Actors in Marathi theatre",
        "Marathi-language writers",
        "Articles containing Marathi-language text",
        "Marathi terms",
        "Marathi-language literature",
        "Marathi-language poets",
        # Newly added categories for Marathi
        "Marathi literature",
        "Marathi cinema",
        "Marathi music",
        "Festivals in Maharashtra",
        "Marathi cuisine",
        "Marathi mythology & Bhakti saints",
        "Marathi theatre (Sangeet Natak)",
        "Notable Marathi people",
        "Marathi media",
        "Awards in Marathi",
    ],
    'or': [
        "Odia language",
        "Articles containing Odia-language text",
        "Odia-language writers",
        "Odia cuisine",
        "Actors in Odia cinema",
        "Odia literature",
        "Odia people",
        "Odia-language films",
        "Odia-language actors",
        "Odia Hindu saints",
        # Newly added categories for Odia
        "Odia music",
        "Festivals of Odisha",
        "Odisha folklore & mythology",
        "Odissi dance and arts",
        "Notable Odia people",
        "Odia media",
        "Odisha awards and organizations",
    ],
    'pa': [
        "Punjabi festivals",
        "Punjabi language",
        "Punjabi people",
        "Punjabi dialects",
        "Punjabi words and phrases",
        "Punjabi music",
        "Punjabi-language films",
        "Punjabi-language writers",
        "Punjabi-language singers",
        "Articles containing Punjabi-language text",
        "Punjabi-language surnames",
        # Newly added categories for Punjabi
        "Punjabi literature",
        "Punjabi cinema",
        "Festivals of Punjab",
        "Punjabi cuisine",
        "Sikh mythology and history",
        "Punjabi folk culture",
        "Notable Punjabi people",
        "Punjabi media",
        "Punjab region history events",
    ],
    'ta': [
        "Tamil",
        "Tamil people",
        "Tamil-language films",
        "Tamil film directors",
        "Tamil diaspora",
        "Tamil film producers",
        "Tamil writers",
        "Articles containing Tamil-language text",
        "Tamil architecture",
        "People from Tamil Nadu",
        "Tamil scholars",
        # Newly added categories for Tamil
        "Tamil literature",
        "Tamil cinema",
        "Tamil music and songs",
        "Tamil festivals",
        "Tamil cuisine",
        "Tamil mythology and religion",
        "Tamil dynasties and history",
        "Notable Tamil people",
        "Tamil media",
        "Tamil theatre and arts",
    ],
    'te': [
        "Telugu people",
        "Telugu-language films",
        "Telugu actors",
        "Telugu cinema",
        "Telugu film directors",
        "Telugu language",
        "Articles containing Telugu-language text",
        "Telugu writers",
        "Telugu poets",
        "Telugu names",
        # Newly added categories for Telugu
        "Telugu literature",
        "Telugu music",
        "Festivals in Andhra/Telangana",
        "Telugu cuisine",
        "Telugu mythology & religion",
        "Telugu theater",
        "Notable Telugu people",
        "Telugu media",
        "Awards in Telugu",
    ],
}

# Path to the root directory of the Indic NLP resources
# Make sure to point this to your local "indic_nlp_resources" folder
INDIC_RESOURCES_PATH = 'D:/indic_nlp_resources-master/indic_nlp_resources-master'
common.set_resources_path(INDIC_RESOURCES_PATH)
# /home/user/112101054_Yash/indic_nlp_resources-master  -- on ubuntu

SIM_THRESHOLDS   = [0.6, 0.7, 0.8]            # τ  – cosine similarity
ALIGN_THRESHOLDS = [0.4, 0.5, 0.6, 0.7, 0.8]  # α  – word‑alignment ratio


###############################################################################
#                              LOAD MODELS
###############################################################################

logging.info("Loading multilingual transformer model (LaBSE)...")
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE')
model     = AutoModel.from_pretrained('sentence-transformers/LaBSE')
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
logging.info(f"Transformer model loaded on device: {device}")

logging.info("Loading SpaCy models...")
try:
    nlp_en = spacy.load('en_core_web_sm')
except Exception as e:
    logging.error(f"Error loading SpaCy English model: {e}")

if "sentencizer" not in nlp_en.pipe_names:
    logging.info("Adding 'sentencizer' to SpaCy English pipeline.")
    nlp_en.add_pipe("sentencizer", first=True)

aligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")

###############################################################################
#                              HELPER FUNCTIONS
###############################################################################

def get_articles_in_category(category_title, lang):
    """
    Fetches all article titles under a Wikipedia category.
    """
    url = f'https://{lang}.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'list': 'categorymembers',
        'cmtitle': f'Category:{category_title}',
        'cmlimit': 'max',
        'format': 'json',
    }
    articles = []
    while True:
        try:
            response = session.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            for page in data['query']['categorymembers']:
                if page['ns'] == 0:  # ns=0 indicates articles (not subcategories or files)
                    articles.append(page['title'])
            if 'continue' in data:
                params['cmcontinue'] = data['continue']['cmcontinue']
            else:
                break
        except requests.exceptions.RequestException as e:
            logging.error(f"Request failed for category '{category_title}' in '{lang}': {e}")
            break
    return articles

def get_langlinks(title, source_lang, target_lang):
    """
    Checks if an article in source_lang has an equivalent article in target_lang.
    Returns the title in target_lang if exists, else None.
    """
    url = f'https://{source_lang}.wikipedia.org/w/api.php'
    params = {
        'action': 'query',
        'titles': title,
        'prop': 'langlinks',
        'lllang': target_lang,
        'format': 'json',
    }
    try:
        response = session.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        pages = data['query']['pages']
        for page_id in pages:
            if 'langlinks' in pages[page_id]:
                return pages[page_id]['langlinks'][0]['*']
    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed for langlinks of page '{title}' from '{source_lang}' to '{target_lang}': {e}")
    return None

def get_wikipedia_page(title, lang):
    """
    Fetches the Wikipedia page content for a given title and language.
    """
    url = f'https://{lang}.wikipedia.org/w/api.php'
    params = {
        'action': 'parse',
        'page': title,
        'prop': 'text',
        'format': 'json',
        'redirects': True,
    }
    try:
        response = session.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        if 'error' in data:
            logging.warning(f"Error fetching page '{title}' in '{lang}': {data['error']['info']}")
            return None
        html_content = data['parse']['text']['*']
        return html_content
    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed for page '{title}' in '{lang}': {e}")
        return None

def extract_text(html_content):
    """
    Extracts and cleans text from HTML content.
    """
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove certain tags which are not part of main text content
    for element in soup(['table', 'sup', 'span', 'ol', 'ul', 'style', 'script']):
        element.decompose()

    text = soup.get_text(separator=' ')
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def split_into_sentences(text, lang):
    """
    Splits text into sentences using appropriate tokenizer based on language.
    """
    if lang == 'en':
        doc = nlp_en(text)
        sentences = [sent.text.strip() for sent in doc.sents]
    else:
        # For Indic languages, use the Indic NLP library
        sentences = sentence_tokenize.sentence_split(text, lang)
    return sentences

def encode_sentences(sentences):
    """
    Encodes sentences into vectors using a multilingual transformer model.
    """
    embeddings = []
    batch_size = 32
    with torch.no_grad():
        for i in range(0, len(sentences), batch_size):
            batch = sentences[i:i+batch_size]
            inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True).to(device)
            outputs = model(**inputs)
            batch_embeddings = outputs.pooler_output.cpu()
            embeddings.append(batch_embeddings)
    if embeddings:
        embeddings = torch.cat(embeddings, dim=0)
    else:
        embeddings = torch.tensor([])
    return embeddings

def compute_similarity(embeddings_en, embeddings_target):
    """
    Computes cosine similarity between English and target language sentence embeddings.
    """
    if embeddings_en.size(0) == 0 or embeddings_target.size(0) == 0:
        logging.warning("One of the embedding sets is empty. Cannot compute similarity.")
        return None

    embeddings_en_np = embeddings_en.numpy()
    embeddings_target_np = embeddings_target.numpy()

    # Normalize each embedding vector
    embeddings_en_norm = embeddings_en_np / np.linalg.norm(embeddings_en_np, axis=1, keepdims=True)
    embeddings_target_norm = embeddings_target_np / np.linalg.norm(embeddings_target_np, axis=1, keepdims=True)

    sim_matrix = cosine_similarity(embeddings_en_norm, embeddings_target_norm)
    return sim_matrix

def align_sentences(sentences_en, sentences_target, sim_matrix, threshold=0.8, alignment_threshold=0.8):
    """
    Aligns sentences based on similarity scores with bidirectional best matching and filtering.
    Uses word alignment from SimAlign to check alignment quality.

    :param sentences_en: list of English sentences
    :param sentences_target: list of target-language sentences
    :param sim_matrix: computed similarity matrix
    :param threshold: minimum similarity score for alignment
    :param alignment_threshold: proportion of aligned words required to confirm alignment
    :return: list of (en_sentence, tgt_sentence, alignment_score, sim_score)
    """
    aligned_sentences = []

    # For each English sentence, find the best target match
    best_matches_en_to_tgt = sim_matrix.argmax(axis=1)
    best_scores_en_to_tgt = sim_matrix.max(axis=1)

    # For each target sentence, find the best English match
    best_matches_tgt_to_en = sim_matrix.argmax(axis=0)
    best_scores_tgt_to_en = sim_matrix.max(axis=0)

    for en_idx, tgt_idx in enumerate(best_matches_en_to_tgt):
        sim_score = best_scores_en_to_tgt[en_idx]
        # Check if the similarity is above threshold
        if sim_score < threshold:
            continue

        # Check if it's a mutual best match
        if best_matches_tgt_to_en[tgt_idx] == en_idx and best_scores_tgt_to_en[tgt_idx] >= threshold:
            en_sentence = sentences_en[en_idx]
            tgt_sentence = sentences_target[tgt_idx]

            # Length filtering
            len_en = len(en_sentence.split())
            len_tgt = len(tgt_sentence.split())
            if min(len_en, len_tgt) == 0:
                continue
            length_ratio = max(len_en, len_tgt) / min(len_en, len_tgt)
            if length_ratio > 1.3:
                continue

            # Word alignment using SimAlign
            alignments = aligner.get_word_aligns(en_sentence, tgt_sentence)
            num_aligned_words = len(alignments['inter'])
            alignment_score = num_aligned_words / min(len_en, len_tgt)

            if alignment_score >= alignment_threshold:
                aligned_sentences.append((en_sentence, tgt_sentence, alignment_score, float(sim_score)))

    return aligned_sentences


###############################################################################
#                              MAIN LOGIC
###############################################################################

def main(
    langs: list[str] | None = None,
    sim_thresholds: list[float] | None = None,
    alpha_thresholds: list[float] | None = None,
):
    # reslove parameter overrides ------------------------------
    if langs is None:
        langs = INDIC_LANGUAGES
    if sim_thresholds is None:
        sim_thresholds = SIM_THRESHOLDS
    if alpha_thresholds is None:
        alpha_thresholds = ALIGN_THRESHOLDS

    EN_LANG = 'en'

    for lang_code in langs:
        language_name = LANG_CODE_MAP.get(lang_code, lang_code)
        logging.info(f"==================== {language_name} ({lang_code}) "
                     "====================")

        #######################################################################
        # Containers for one single crawl pass
        #######################################################################
        all_pairs         = []      # every (sentence, sim, align, category)
        article_category  = {}      # article_title -> first English category
        #######################################################################

        # ------------- 1. COLLECT ARTICLE TITLES (unchanged + mapping) ------
        categories = language_categories.get(lang_code, [])
        if not categories:
            logging.warning(f"No categories specified for {language_name}. Skipping.")
            continue

        article_titles = set()
        for cat in categories:
            titles_en = get_articles_in_category(cat, EN_LANG)
            for t in titles_en:
                article_category.setdefault(t, cat)   # store only first cat
            article_titles.update(titles_en)

        total_articles_fetched = len(article_titles)
        logging.info(f"Found {total_articles_fetched} unique English articles.")

        # ------ 2. PROCESS EACH BILINGUAL ARTICLE (download & align once) ----
        titles_with_langlinks = 0
        for title in tqdm(article_titles,
                          desc=f"Langlinks > {language_name}",
                          leave=False):
            tgt_title = get_langlinks(title, EN_LANG, lang_code)
            if tgt_title is None:
                continue
            titles_with_langlinks += 1

            html_en  = get_wikipedia_page(title,     EN_LANG)
            html_tgt = get_wikipedia_page(tgt_title, lang_code)
            if not html_en or not html_tgt:
                continue

            text_en  = extract_text(html_en)
            text_tgt = extract_text(html_tgt)
            if not text_en or not text_tgt:
                continue

            # ---- sentence split & length filter ----------------------------
            def filter_sents(sents, lo=5, hi=200):
                return [s for s in sents if lo <= len(s.split()) <= hi]

            sents_en  = filter_sents(split_into_sentences(text_en,  EN_LANG))
            sents_tgt = filter_sents(split_into_sentences(text_tgt, lang_code))
            if not sents_en or not sents_tgt:
                continue

            # ---- embed once -------------------------------------------------
            emb_en  = encode_sentences(sents_en)
            emb_tgt = encode_sentences(sents_tgt)
            sim_mtx = compute_similarity(emb_en, emb_tgt)
            if sim_mtx is None:
                continue

            # ---- capture all potential pairs (τ=0, α=0) ----------------
            aligned_raw = align_sentences(
                sents_en, sents_tgt, sim_mtx,
                threshold=0.4, alignment_threshold=0.0
            )

            cat = article_category[title]
            for (sen_en, sen_tgt, align_sc, sim_sc) in aligned_raw:
                all_pairs.append({
                    "en": sen_en,
                    "tgt": sen_tgt,
                    "sim": sim_sc,
                    "align": align_sc,
                    "category": cat
                })
        logging.info(f"{titles_with_langlinks}/{total_articles_fetched} "
                     f"articles had {language_name} counterparts.")
        logging.info(f"Total raw sentence pairs harvested: {len(all_pairs)}")

        # ---------- 3. MULTI‑THRESHOLD OUTPUT LOOP --------------------------
        for tau in sim_thresholds:
            for alpha in alpha_thresholds:
                # Directory structure:  parallel_datasets/<τ>/<α>/…
                ds_dir = os.path.join("parallel_datasets", str(tau), str(alpha))
                st_dir = os.path.join("stats",             str(tau), str(alpha))
                os.makedirs(ds_dir, exist_ok=True)
                os.makedirs(st_dir, exist_ok=True)

                # Filter pairs that satisfy the current grid point
                selected = [
                    p for p in all_pairs
                    if p["sim"] >= tau and p["align"] >= alpha
                ]

                # ---------- PARALLEL DATASET JSONL --------------------------
                df = pd.DataFrame(
                    [(p["en"], p["tgt"]) for p in selected],
                    columns=["English", language_name]
                )
                ds_path = os.path.join(
                    ds_dir, f"parallel_dataset_{language_name.lower()}.json"
                )
                df.to_json(ds_path, orient="records",
                           lines=True, force_ascii=False)

                # ---------- STATS (aggregate + per‑category) ---------------
                agg = {
                    "language": language_name,
                    "tau": tau,
                    "alpha": alpha,
                    "num_pairs": len(selected),
                    "mean_cosine": (float(np.mean([p["sim"]
                                      for p in selected]))
                                    if selected else 0.0),
                    "mean_alignment": (float(np.mean([p["align"]
                                         for p in selected]))
                                       if selected else 0.0)
                }
                by_cat = {}
                for cat in categories:
                    cp = [p for p in selected if p["category"] == cat]
                    if cp:
                        by_cat[cat] = {
                            "num_pairs": len(cp),
                            "mean_cosine": float(np.mean([p["sim"] for p in cp])),
                            "mean_alignment": float(np.mean([p["align"]
                                                             for p in cp]))
                        }

                st_path = os.path.join(
                    st_dir, f"stats_{language_name.lower()}.json"
                )
                with open(st_path, "w", encoding="utf-8") as f:
                    json.dump({"aggregate": agg, "by_category": by_cat},
                              f, ensure_ascii=False, indent=4)

                logging.info(f"[{language_name}] τ={tau} α={alpha} "
                             f"→ {len(selected)} pairs "
                             f"(saved: {ds_path})")

# --------------------------------------------------------------------------- #
if __name__ == "__main__":
    #default execution
    main()

    # run just Hindi with one grid point
    # main(
    #     langs=['hi'],
    #     sim_thresholds=[0.7],    # τ
    #     alpha_thresholds=[0.6]   # α
    # )

    # # run Bengali + Gujarati, full default grids
    # main(langs=['bn', 'gu'])

    # # run everything but with a finer τ grid
    # main(sim_thresholds=[0.55, 0.6, 0.7, 0.8])


    # # Should run one small grid without errors
    # main(langs=['as'], sim_thresholds=[0.7], alpha_thresholds=[0.6])


In [ ]:
# ------------------------------------------------------------
# 1.  Imports & setup
# ------------------------------------------------------------
import json, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

STATS_ROOT = "stats"
TAU_GRID   = [0.6, 0.7, 0.8]
ALPHA_GRID = [0.4, 0.5, 0.6, 0.7, 0.8]
LANGS      = ["as","bn","gu","hi","kn","ml","mr","or","pa","ta","te"]
LANG_FULL = {
    'as':'Assamese','bn':'Bengali','gu':'Gujarati','hi':'Hindi',
    'kn':'Kannada','ml':'Malayalam','mr':'Marathi','or':'Oriya',
    'pa':'Punjabi','ta':'Tamil','te':'Telugu'
}

def load_stats(lang, tau, alpha):
    path = os.path.join(STATS_ROOT, str(tau), str(alpha),
                        f"stats_{LANG_FULL[lang].lower()}.json")
    if not os.path.exists(path):
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# ------------------------------------------------------------
# 2.  Sweet‑spot heat‑map (robust to missing files)
# ------------------------------------------------------------
fig, axes = plt.subplots(4, 3, figsize=(15,20))
axes = axes.flatten()
cmap = plt.get_cmap('plasma')

for idx, lang in enumerate(LANGS):
    grid_cos = np.zeros((len(ALPHA_GRID), len(TAU_GRID)))
    grid_cnt = np.zeros_like(grid_cos, dtype=int)

    for i, alpha in enumerate(ALPHA_GRID):
        for j, tau in enumerate(TAU_GRID):
            stats = load_stats(lang, tau, alpha)
            if stats:
                grid_cos[i,j] = stats["aggregate"]["mean_cosine"]
                grid_cnt[i,j] = stats["aggregate"]["num_pairs"]
            else:
                grid_cos[i,j] = np.nan
                grid_cnt[i,j] = 0

    ax = axes[idx]
    im = ax.imshow(grid_cos, aspect="auto", cmap=cmap, vmin=0.5, vmax=1.0)
    ax.set_xticks(range(len(TAU_GRID))); ax.set_xticklabels(TAU_GRID, rotation=45)
    ax.set_yticks(range(len(ALPHA_GRID))); ax.set_yticklabels(ALPHA_GRID)
    ax.set_title(LANG_FULL[lang], pad=10)

    for i in range(len(ALPHA_GRID)):
        for j in range(len(TAU_GRID)):
            val = grid_cos[i,j]
            count = grid_cnt[i,j]
            if np.isnan(val):
                txt = "–\n0"
                color = "grey"
            else:
                txt = f"{val:.2f}\n{count}"
                color = 'white' if val<0.75 else 'black'
            ax.text(j, i, txt, ha="center", va="center", fontsize=8, color=color)

fig.subplots_adjust(right=0.85, top=0.92)
cbar_ax = fig.add_axes([0.88,0.15,0.02,0.7])
fig.colorbar(im, cax=cbar_ax, label='Mean Cosine')
fig.suptitle("Sweet‑spot Heatmap", fontsize=16)
plt.tight_layout(rect=[0,0,0.85,0.96])
plt.show()

# ------------------------------------------------------------
# 3.  Category bar‑chart for τ=0.7, α=0.6
# ------------------------------------------------------------
SAMPLE_TAU, SAMPLE_ALPHA = 0.7, 0.6
for lang in LANGS:
    stats = load_stats(lang, SAMPLE_TAU, SAMPLE_ALPHA)
    if not stats or not stats.get("by_category"):
        continue
    df_cat = pd.DataFrame.from_dict(stats["by_category"], orient="index")
    df_cat["num_pairs"] = df_cat.get("num_pairs", 0)
    df_cat = df_cat.sort_values("num_pairs", ascending=False).head(5)
    plt.figure(figsize=(8,4))
    plt.barh(df_cat.index[::-1], df_cat["num_pairs"][::-1], edgecolor='k')
    plt.title(f"Top‑5 Categories: {LANG_FULL[lang]} (τ={SAMPLE_TAU}, α={SAMPLE_ALPHA})", pad=8)
    plt.xlabel("Sentence Pairs")
    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------
# 4.  Executive table for τ=0.7, α=0.6
# ------------------------------------------------------------
rows = []
for lang in LANGS:
    stats = load_stats(lang, SAMPLE_TAU, SAMPLE_ALPHA)
    if not stats:
        rows.append({"Language": LANG_FULL[lang], "Pairs":0, "Mean Cos":0, "Mean Align":0})
    else:
        agg = stats["aggregate"]
        rows.append({
            "Language": LANG_FULL[lang],
            "Pairs":     agg["num_pairs"],
            "Mean Cos":  round(agg["mean_cosine"],3),
            "Mean Align":round(agg["mean_alignment"],3)
        })
exec_df = pd.DataFrame(rows).sort_values("Pairs", ascending=False).reset_index(drop=True)
display(exec_df)        # Jupyter/Colab will render as a neat table

# also save as CSV if you like
exec_df.to_csv("executive_table_0.6_0.7.csv", index=False)